In [2]:
import numpy as np
import copy
import pygame

pygame 2.1.2 (SDL 2.0.18, Python 3.9.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
directions = (
[[0,   1,   0],
[0,   0,   0],         #North
[0,   0,   0]],

[
[0,   0,   0],
[0,   0,   0],         # South
[0,   1,   0]],

[
[0,   0,   0],
[0,   0,   1],         # East
[0,   0,   0]],

[
[0,   0,   0],
[1,   0,   0],         # West
[0,   0,   0]]
)

In [45]:
default_grid = [
    [(0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0)],
    [(0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0)],
    [(0,0), (0,0), (0,0), (0,0), (-100,"r"), (-100,"r"), (-100,"r"), (-100,"r"), (0,0), (-100,"r")],
    [(0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0)],
    [(0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0)],
    [(0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0)],
    [(0,0), (0,0), (-100,"r"), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0)],
    [(0,0), (0,0), (-100,"r"), (-100,"r"), (0,0), (0,0), (-100,"r"), (-100,"r"), (-100,"r"), (0,0)],
    [(0,0), (0,0), (-100,"r"), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0)],
    [(0,0), (0,0), (-100,"r"), (-100,"r"), (-100,"r"), (-100,"r"), (-100,"r"), (-100,"r"), (-100,"r"), (100,"r")]
        ]

In [46]:
def initialize_grid():
    grid = []
    for row in range(10): #10 Rows 
        blocks_in_row = [] #Empty Row list
        for column in range(10):
            blocks_in_row.append((0, 0)) #Appending block values
        grid.append(blocks_in_row)
    return grid

def place_objects(grid:list):
    for i in range(np.random.randint(0, 10)):
        grid[np.random.randint(0, 9)][np.random.randint(0, 9)] = (-100, "r") #Randomly placing red blocks

    for i in range(np.random.randint(0, 7)):
        grid[np.random.randint(0, 9)][np.random.randint(0, 9)] = (100, "r") #Randomly placing green blocks
    return grid

In [47]:
def calculate_value_direction(position, grid, direction):
    value = 0
    for i in [-1,1]:
        if position[0]+i >= 0 and position[0]+i < 10 : 
            value += grid[position[0]+i][position[1]][0] * direction[1+i][1]
        else:
            value += grid[position[0]][position[1]][0] * direction[1+i][1]

        if position[1]+i >= 0 and position[1]+i < 10:
            value += grid[position[0]][position[1]+i][0] * direction[1][1+i]
        else:
            value += grid[position[0]][position[1]][0] * direction[1][1+i]
            
    return value

In [48]:
def calc_values(position, grid, gamma):

    if grid[position[0]][position[1]][1] == "r":
        return (grid[position[0]][position[1]],)*4

    value = {}
    value['north'] = calculate_value_direction(position, grid, N) * gamma
    value['south'] = calculate_value_direction(position, grid, S) * gamma
    value['east'] = calculate_value_direction(position, grid, E) * gamma
    value['west'] = calculate_value_direction(position, grid, W) * gamma
    
    return ((value['north'],"north"), (value['east'],"east"), (value['west'],"west"), (value['south'],"south"))

In [49]:
def calc_arrow_coordinates(direction, row, col):
    if direction == "north":
        return (
        (col*(MARGIN + HEIGHT)+14, row*(MARGIN + WIDTH)+22), 
        (col*(MARGIN + HEIGHT)+16, row*(MARGIN + WIDTH)+22), 
        (col*(MARGIN + HEIGHT)+16, row*(MARGIN + WIDTH)+15), 
        (col*(MARGIN + HEIGHT)+20, row*(MARGIN + WIDTH)+15), 
        (col*(MARGIN + HEIGHT)+15, row*(MARGIN + WIDTH)+6 ), 
        (col*(MARGIN + HEIGHT)+10, row*(MARGIN + WIDTH)+15), 
        (col*(MARGIN + HEIGHT)+14, row*(MARGIN + WIDTH)+15))
    
    elif direction == "south":
        return (
        (col*(MARGIN + HEIGHT)+14, row*(MARGIN + WIDTH)+6 ), 
        (col*(MARGIN + HEIGHT)+16, row*(MARGIN + WIDTH)+6 ), 
        (col*(MARGIN + HEIGHT)+16, row*(MARGIN + WIDTH)+15), 
        (col*(MARGIN + HEIGHT)+20, row*(MARGIN + WIDTH)+15), 
        (col*(MARGIN + HEIGHT)+15, row*(MARGIN + WIDTH)+22), 
        (col*(MARGIN + HEIGHT)+10, row*(MARGIN + WIDTH)+15), 
        (col*(MARGIN + HEIGHT)+14, row*(MARGIN + WIDTH)+15))

    elif direction == "east":
        return (
        (col*(MARGIN + WIDTH)+6 , row*(MARGIN + HEIGHT)+14), 
        (col*(MARGIN + WIDTH)+6 , row*(MARGIN + HEIGHT)+16), 
        (col*(MARGIN + WIDTH)+15, row*(MARGIN + HEIGHT)+16), 
        (col*(MARGIN + WIDTH)+15, row*(MARGIN + HEIGHT)+20), 
        (col*(MARGIN + WIDTH)+22, row*(MARGIN + HEIGHT)+15), 
        (col*(MARGIN + WIDTH)+15, row*(MARGIN + HEIGHT)+10), 
        (col*(MARGIN + WIDTH)+15, row*(MARGIN + HEIGHT)+14))

    elif direction == "west":
        return (
        (col*(MARGIN + WIDTH)+22, row*(MARGIN + HEIGHT)+14), 
        (col*(MARGIN + WIDTH)+22, row*(MARGIN + HEIGHT)+16), 
        (col*(MARGIN + WIDTH)+15, row*(MARGIN + HEIGHT)+16), 
        (col*(MARGIN + WIDTH)+15, row*(MARGIN + HEIGHT)+20), 
        (col*(MARGIN + WIDTH)+6 , row*(MARGIN + HEIGHT)+15), 
        (col*(MARGIN + WIDTH)+15, row*(MARGIN + HEIGHT)+10), 
        (col*(MARGIN + WIDTH)+15, row*(MARGIN + HEIGHT)+14))
    

In [50]:
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

WIDTH = 20
HEIGHT = 20
MARGIN = 5

In [51]:
N, S, E, W = directions #initializing moveable directions
def avengers_assemble(default):
    alpha = 0.9 #learning rate

    if default:
        grid = default_grid
    else:
        grid = place_objects(initialize_grid()) #creating a random grid 
    

    shadow = []

    while(grid != shadow):

        shadow = copy.deepcopy(grid)

        for i in range(10):
            for j in range(10):
                n, e, w, s = calc_values((i,j), shadow, alpha)
                grid[i][j] = max(([n,e,w,s][::-1]) , key = lambda x: x[0])


    pygame.init()

    # Set the HEIGHT and WIDTH of the screen
    WINDOW_SIZE = [255, 255]
    screen = pygame.display.set_mode(WINDOW_SIZE)

    # Set title of screen
    pygame.display.set_caption("Reinforcement Learning")

    # Loop until the user clicks the close button.
    done = False

    while not done:
        for event in pygame.event.get():  # User did something
            if event.type == pygame.QUIT:  # If user clicked close
                done = True

        for row in range(len(grid)):
            for col in range(len(grid[0])):
                color = WHITE

                if grid[row][col][1] == "r":

                    if grid[row][col][0] > 0 :
                        color = GREEN
                    elif grid[row][col][0] < 0 :
                        color = RED
                        
                pygame.draw.rect(screen, color, [(MARGIN + WIDTH) * col + MARGIN,
                                (MARGIN + HEIGHT) * row + MARGIN, WIDTH, HEIGHT])

                if grid[row][col][1] != "r":
                    pygame.draw.polygon(screen, (0, 0, 0), calc_arrow_coordinates(grid[row][col][1], row, col))

        pygame.display.flip()


    pygame.quit()

In [54]:
avengers_assemble(0) #random grid
# avengers_assemble(1) #default grid

Note: Pygame assistance taken from previous submissions